In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df=pd.read_csv('/content/drive/My Drive/ML_PROJECTS/Movie_reccomend/main_data.csv')
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...


Since we are going to use count vectroizer , the person's having same first name can lead to some similarity.
Removing the spaces between the names

In [3]:
def rem_space(x):
  x=x.replace(' ','')
  return x

In [4]:
df['director_name']=df['director_name'].map(lambda x:rem_space(x))
df['actor_1_name']=df['actor_1_name'].map(lambda x:rem_space(x))
df['actor_2_name']=df['actor_2_name'].map(lambda x:rem_space(x))
df['actor_3_name']=df['actor_3_name'].map(lambda x:rem_space(x))
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,JamesCameron,CCHPounder,JoelDavidMoore,WesStudi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,GoreVerbinski,JohnnyDepp,OrlandoBloom,JackDavenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,SamMendes,ChristophWaltz,RoryKinnear,StephanieSigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,ChristopherNolan,TomHardy,ChristianBale,JosephGordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,DougWalker,DougWalker,RobWalker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...


In [5]:
df.drop(columns=['comb'],inplace=True,axis=1)
df['comb']=df['director_name'] +' ' + df['actor_1_name'] + ' ' + df['actor_2_name'] + ' '+ df['actor_3_name'] + ' ' + df['genres']

In [6]:
df['comb']=df['comb'].str.lower()
#df.to_csv('/content/drive/My Drive/ML_PROJECTS/Movie_reccomend/last.csv')

Setting inde to movie name so it is easy to get movie details with the help of movie name

In [7]:
df.set_index(df['movie_title'],inplace=True)

Using count vectrozier

In [8]:
vectorizer = CountVectorizer()
mat=vectorizer.fit_transform(df['comb'])

Findong cosine similarity(measures the angle between the vectors) between movies

In [9]:
cosine_sim=cosine_similarity(mat,mat)

In [10]:
sim=pd.DataFrame(cosine_sim)
sim.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6076,6077,6078,6079,6080,6081,6082,6083,6084,6085,6086,6087,6088,6089,6090,6091,6092,6093,6094,6095,6096,6097,6098,6099,6100,6101,6102,6103,6104,6105,6106,6107,6108,6109,6110,6111,6112,6113,6114,6115
0,1.000000,0.377964,0.251976,0.125988,0.0,0.471405,0.201008,0.471405,0.235702,0.444444,0.471405,0.272166,0.377964,0.251976,0.555556,0.353553,0.377964,0.502519,0.272166,0.377964,0.235702,0.272166,0.251976,0.136083,0.471405,0.444444,0.444444,0.353553,0.471405,0.527046,0.222222,0.471405,0.471405,0.555556,0.111111,0.471405,0.222222,0.471405,0.125988,0.421637,...,0.0,0.0,0.235702,0.0,0.117851,0.0,0.0,0.235702,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.125988,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.333333,0.125988,0.000000,0.117851,0.000000,0.0,0.377964,0.0,0.235702,0.0,0.000000,0.0
1,0.377964,1.000000,0.285714,0.142857,0.0,0.267261,0.227921,0.267261,0.267261,0.251976,0.267261,0.308607,1.000000,0.571429,0.377964,0.400892,0.571429,0.341882,0.308607,0.428571,0.267261,0.308607,0.285714,0.000000,0.267261,0.251976,0.251976,0.400892,0.267261,0.358569,0.251976,0.267261,0.267261,0.377964,0.125988,0.267261,0.251976,0.267261,0.142857,0.239046,...,0.0,0.0,0.267261,0.0,0.133631,0.0,0.0,0.267261,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.142857,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.125988,0.0,0.377964,0.142857,0.000000,0.133631,0.000000,0.0,0.428571,0.0,0.267261,0.0,0.000000,0.0
2,0.251976,0.285714,1.000000,0.285714,0.0,0.267261,0.113961,0.267261,0.133631,0.251976,0.267261,0.462910,0.285714,0.285714,0.251976,0.267261,0.285714,0.227921,0.154303,0.285714,0.267261,0.154303,0.142857,0.000000,0.267261,0.377964,0.377964,0.267261,0.267261,0.358569,0.125988,0.267261,0.267261,0.251976,0.125988,0.267261,0.125988,0.267261,0.285714,0.358569,...,0.0,0.0,0.133631,0.0,0.000000,0.0,0.0,0.133631,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.142857,0.0,0.0,0.133631,0.0,0.0,0.0,0.0,0.0,0.251976,0.0,0.251976,0.285714,0.169031,0.133631,0.154303,0.0,0.285714,0.0,0.133631,0.0,0.142857,0.0
3,0.125988,0.142857,0.285714,1.000000,0.0,0.133631,0.000000,0.133631,0.000000,0.125988,0.133631,0.154303,0.142857,0.142857,0.125988,0.133631,0.142857,0.113961,0.000000,0.142857,0.133631,0.000000,0.000000,0.000000,0.133631,0.251976,0.251976,0.133631,0.133631,0.239046,0.000000,0.133631,0.133631,0.125988,0.000000,0.133631,0.000000,0.267261,0.285714,0.239046,...,0.0,0.0,0.133631,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.142857,0.0,0.0,0.133631,0.0,0.0,0.0,0.0,0.0,0.251976,0.0,0.125988,0.285714,0.169031,0.133631,0.154303,0.0,0.142857,0.0,0.133631,0.0,0.142857,0.0
4,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.133631,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


Given a movie name we are finding the index and then taking the row of that index from out sim dataframe and then taking the first 10 values(except the 1st value) that are most similar 

In [11]:
index=pd.Series(df.index)
def reccomend(name):
  
  idx=index[index==name].index[0]
  row=pd.Series(sim.iloc[:,idx])
  row=row.sort_values(ascending=False)
  row=row[1:11].index

  recc=[]
  for i in row:
    recc.append(list(df.index)[i])
  return recc

In [12]:
reccomend('the dark knight rises')

['inception',
 'the dark knight',
 'sin city: a dame to kill for',
 'premium rush',
 'miracle at st. anna',
 'g.i. joe: the rise of cobra',
 'batman begins',
 'looper',
 'rocknrolla',
 'the juror']